# **COMP 2211 Exploring Artificial Intelligence** #
## Lab 9 Minimax and Alpha-Beta Pruning ##

## **Lab tasks & Submission scheme**

**Lab tasks:**

In this lab, you are required to implement a game, Simple-Nim, using Minimax and Alpha-Beta Pruning.

 **Submission to ZINC:**

 After completing all [TODO] in this notebook, successfully running the code, and achieving the task requirement, please **COPY** all your [TODO] to the template __lab9_task.py__ (do not change the file name). Then, zip __lab9_task.py__ and submit **ONLY** the .zip file to ZINC.

 Before you submit your codes, please check the ***indentations***.

## **Connect-Four Games**

![connect4.png](attachment:connect4.png)


Connect-4 is a strategy game played between two players who take turns dropping colored discs into a vertically suspended grid. The objective is to be the first to form a line of four discs of one's own color, either horizontally, vertically, or diagonally. Here is an example of how a game might progress:

**Setting**
* 7x6 Grid:

  🟨🟨🟨🟨🟨🟨🟨

  🟨🟨🟨🟨🟨🟨🟨

  🟨🟨🟨🟨🟨🟨🟨

  🟨🟨🟨🟨🟨🟨🟨

  🟨🟨🟨🟨🟨🟨🟨

  🟨🟨🟨🟨🟨🟨🟨

* Player 1--Human (Red Discs): 😆

* Player 2--AI (Blue Discs): 😎

**Turn Examples**
1. 😆: Drop a red disc in column 4

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟥🟨🟨🟨

2. 😎: Drop a blue disc in column 3

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟦🟥🟨🟨🟨

3. 😆: Drop a red disc in column 4

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟨🟨🟨🟨

   🟨🟨🟨🟥🟨🟨🟨

   🟨🟨🟦🟥🟨🟨🟨
   
...and so on until one player aligns four discs of their color.

**Winner:** The first player to line up four of their discs (either 😆 or 😎) wins the game!

(You can play a demo of Connect-4 [here](https://www.mathsisfun.com/games/connect4.html))


## **Minimax algorithm for the Connect-4 Game**

In this task, you will implement a minimax algorithm for the Connect-4 game.


### General Flow:

1.      Initialization: The game starts by initializing the board.
2.      Gameplay: Players (or AI and player) take turns to drop pieces on the board.
3.      Checking States: After each move, the game checks for a win or tie.
4.      AI Decision: If it's AI's turn, it calculates the best move based on the current board state.
5.      End of Game: The game ends when a player wins or the board fills up (tie).

### Before we begin

We first need to build the basic class of Connect-4 games.

#### Implementing the basic class of Connect-4 Games


The steps are as follows:
1.   set the gaming parameters.
2.   Game Mechanics: build the basic classes, including the following methods:
        - ***create_board***
        - ***drop_piece***
        - ***is_valid_location***
        - ***get_next_open_row***
3.   Win Condition Checks
4.   AI Decision Making
        - ***evaluate_window***
        - ***score_position***
        - ***get_valid_locations***
        - ***pick_best_move***
5.   Terminal State Check





### 1. Set the gaming parameters

Set the dimensions of the game board and define constants for players, pieces, and window length for checking win conditions.


```

        # size of gaming board
        ROW_COUNT = 6
        COLUMN_COUNT = 7
        # denote which one to play
        PLAYER = 0
        AI = 1
        # use for initialize the gaming board
        EMPTY = 0
        # specific piece number of  Player and AI
        PLAYER_PIECE = 1
        AI_PIECE = 2
        WINDOW_LENGTH = 4
```

### 2. Game Mechanics


*create_board*:

This method is used to initialize the Connect-4 game, to create a 6x7 board filled with zeros, representing an empty board.


*drop_piece*:

This method is used to place a piece on the board at a specified location. It's a crucial part of the game mechanics, allowing players or AI to make moves.

*is_valid_location*:

This method checks if a move is valid. It ensures that a piece is dropped in a column that is not already full.

*get_next_open_row*:

After determining a valid column, this method finds the next open row within that column where the piece will actually be placed, simulating the dropping of a piece in Connect-4.

In [1]:
import numpy as np
## Default parameters for this lab

# size of gaming board
ROW_COUNT = 6
COLUMN_COUNT = 7

# denote which one to play
PLAYER = 0
AI = 1

# use for initialize the gaming board
EMPTY = 0

# specific piece number of  Player and AI
PLAYER_PIECE = 1
AI_PIECE = 2

WINDOW_LENGTH = 4

class ConnectFour:

    #Initialize a 7x6 board with all empty slots.
    def create_board(self):
        board = np.zeros((ROW_COUNT,COLUMN_COUNT))
        return board

    # drop a piece into the specific (row, column)
    def drop_piece(self, board, row, col, piece):
        board[row][col] = piece

    # check current position in the board is valid
    # Checks if a move is valid by ensuring the top row of the specified column is empty.
    def is_valid_location(self, board, col):
        return board[0][col] == 0

    ## This method finds the next open row in a given column where a piece can be dropped.
    # It iterates through each row in the specified column and returns the row number
    # where the first empty space (0) is found.

    def get_next_open_row(self, board, col):
        for r in range(ROW_COUNT):
            if board[r][col] == 0:
                return r

#### 3.Win Condition Checks
*winning_move*:

This methodtion checks whether the current player has won the game. It examines the board for any horizontal, vertical, or diagonal line of four of the same pieces, which is the winning condition in Connect-4.

In [2]:
def winning_move(self, board, piece):
        # Check horizontal locations for win
        for c in range(COLUMN_COUNT-3):
            for r in range(ROW_COUNT):
                if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                    return True

        # Check vertical locations for win
        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT-3):
                if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                    return True

        # Check positively sloped diaganols
        for c in range(COLUMN_COUNT-3):
            for r in range(ROW_COUNT-3):
                if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                    return True

        # Check negatively sloped diaganols
        for c in range(COLUMN_COUNT-3):
            for r in range(3, ROW_COUNT):
                if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                    return True

##### 4.AI Decision Making
*evaluate_window*:

Used by the AI to evaluate a segment of the board (a 'window') and score it based on the arrangement of the pieces. This methodtion is vital for the AI's strategy, as it helps in assessing the potential of a given board situation.

*score_position*:

This methodtion evaluates the entire board from the AI's perspective. It uses evaluate_window to score different parts of the board, aiding the AI in understanding the overall state of the game and making strategic decisions.

> **There are two parts you need to implement.**

☑️ 　　**Score Horizontal**:  Scans each row of the board, and count the scores. \
☑️ 　　**Score Vertical**: Scans each column of the board, and count the scores. \
☑️ 　　**[TODO 1: Score positive sloped diagonal]**: Evaluates diagonal lines sloping upwards, moving through the board, creating diagonal windows and scoring them. \
☑️ 　　**[TODO 2: Negative sloped diagonal]**: Evaluates diagonals sloping downwards, moving through the board, creating diagonal windows and scoring them.


*get_valid_locations*:

It provides the AI with information about which columns are available for dropping a piece, which is essential for deciding the next move.

*pick_best_move*:

This methodtion represents the core of the AI's decision-making process. It iterates through valid locations, simulates moves, and uses score_position to choose the best possible move based on the current state of the board.


> ***To avoid potential debugging issues in the future, please do not change the scoring parameters in the following codes.***

In [3]:
def evaluate_window(self, window, piece):

        # you've provided is designed to score a given window (a subset of the Connect-4 board) based on the presence of player pieces,
        # AI pieces, and empty spaces.

    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE

    # If the window contains four of the piece (either player or AI), the score increases by 100, indicating a winning condition.
    if window.count(piece) == 4:
        score += 100

    # If there are three piece and one empty space, the score increases by 5, indicating a potential win in the next move.
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5

    # If the window has two piece and two empty spaces, the score increases by 2, signifying a developing opportunity.
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    # If the opponent has three pieces and there's one empty space, the score decreases by 4. This reflects the need to block the opponent's potential win.
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(self, board, piece):
    score = 0

    ## Score center column
    ## This part focuses on the center column, often a strategic position in Connect-4.
    # It counts the number of piece in the center column and multiplies this count by 3,
    # adding this to the total score.
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    ## Score Horizontal
    ## This loop scans each row of the board. For each row, it creates a 'window' of 4 spaces
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += self.evaluate_window(window, piece)

    ## Score Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += self.evaluate_window(window, piece)

    '''
    Score positive sloped diagonal
    Here, it evaluates diagonal lines sloping upwards.
    It moves through the board, creating diagonal windows and scoring them.
    '''
    ## [TODO-1]
    for x in range(ROW_COUNT - 3):
        for y in range(COLUMN_COUNT - 3):
            window = [board[x + i][y + i] for i in range(WINDOW_LENGTH)]
            score = score + self.evaluate_window(window, piece)

    '''
    This part evaluates diagonals sloping downwards,
    again using evaluate_window to score these sections.
    '''
    ## [TODO-2]
    for h in range(ROW_COUNT - 3):
        for k in range(COLUMN_COUNT - 3):
            window = [board[h + 3 - i][k + i] for i in range(WINDOW_LENGTH)]
            score = score + self.evaluate_window(window, piece)

def get_valid_locations(self, board):
        valid_locations = []
        for col in range(COLUMN_COUNT):
            if self.is_valid_location(board, col):
                valid_locations.append(col)
        return valid_locations

    # find the best move of AI/Player according to the current board
def pick_best_move(self, board, piece):

    valid_locations = self.get_valid_locations(board)
    best_score = -10000
    best_col = random.choice(valid_locations)
    for col in valid_locations:
        row = self.get_next_open_row(board, col)
        temp_board = board.copy()
        self.drop_piece(temp_board, row, col, piece)
        score = self.score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col

    return best_col

### 5.Terminal State Check
*is_terminal_node*:

This methodtion checks if the game has reached a terminal state, which could be a win for either player, a tie (no more valid moves), or reaching a certain depth limit in more advanced AI implementations.

In [4]:
 # check the gaming is over: win/lose/tie
def is_terminal_node(self, board):
    return self.winning_move(board, PLAYER_PIECE) or self.winning_move(board, AI_PIECE) or len(self.get_valid_locations(board)) == 0


## Basic Connect-4 Class

In [5]:
import numpy as np
import random
import sys
import math


## Default parameters for this lab

# size of gaming board
ROW_COUNT = 6
COLUMN_COUNT = 7

# denote which one to play
PLAYER = 0
AI = 1

# use for initialize the gaming board
EMPTY = 0

# specific piece number of  Player and AI
PLAYER_PIECE = 1
AI_PIECE = 2

WINDOW_LENGTH = 4


class ConnectFour:
    def __init__(self):
        self.board = self.create_board()
        self.last_move = None  # Keeps track of the last move made

    #Initialize a 7x6 board with all empty slots.
    def create_board(self):
        board = np.zeros((ROW_COUNT,COLUMN_COUNT))
        return board

    # drop a piece into the specific (row, column)
    def drop_piece(self, board, row, col, piece):
        board[row][col] = piece

    # check current position in the board is valid
    def is_valid_location(self, board, col):
        return board[0][col] == 0



    ## This method finds the next open row in a given column where a piece can be dropped.
    # It iterates through each row in the specified column and returns the row number
    # where the first empty space (0) is found.

    def get_next_open_row(self, board, col):
        for r in range(ROW_COUNT-1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    # print the board
    def print_board(self, board):
        print("\n".join(["\t".join([str(cell) for cell in row]) for row in board]))

    # check if the current player has won the game.
    def winning_move(self, board, piece):
        # Check horizontal locations for win
        for c in range(COLUMN_COUNT-3):
            for r in range(ROW_COUNT):
                if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                    return True

        # Check vertical locations for win
        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT-3):
                if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                    return True

        # Check positively sloped diaganols
        for c in range(COLUMN_COUNT-3):
            for r in range(ROW_COUNT-3):
                if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                    return True

        # Check negatively sloped diaganols
        for c in range(COLUMN_COUNT-3):
            for r in range(3, ROW_COUNT):
                if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                    return True

    def evaluate_window(self, window, piece):

        # you've provided is designed to score a given window (a subset of the Connect-4 board) based on the presence of player pieces,
        # AI pieces, and empty spaces.

        score = 0
        opp_piece = PLAYER_PIECE
        if piece == PLAYER_PIECE:
            opp_piece = AI_PIECE

        # If the window contains four of the piece (either player or AI), the score increases by 100, indicating a winning condition.
        if window.count(piece) == 4:
            score += 100

        # If there are three piece and one empty space, the score increases by 5, indicating a potential win in the next move.
        elif window.count(piece) == 3 and window.count(EMPTY) == 1:
            score += 5

        # If the window has two piece and two empty spaces, the score increases by 2, signifying a developing opportunity.
        elif window.count(piece) == 2 and window.count(EMPTY) == 2:
            score += 2

        # If the opponent has three pieces and there's one empty space, the score decreases by 4. This reflects the need to block the opponent's potential win.
        if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
            score -= 4

        return score

    # The score_position method you provided is designed to evaluate the entire Connect-4 board and assign a score based on the current position of the pieces.
    # This score helps the AI to make decisions.
    def score_position(self, board, piece):
        score = 0

        ## Score center column
        ## This part focuses on the center column, often a strategic position in Connect-4.
        # It counts the number of piece in the center column and multiplies this count by 3,
        # adding this to the total score.
        center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
        center_count = center_array.count(piece)
        score += center_count * 3

        ## Score Horizontal
        ## This loop scans each row of the board. For each row, it creates a 'window' of 4 spaces
        for r in range(ROW_COUNT):
            row_array = [int(i) for i in list(board[r,:])]
            for c in range(COLUMN_COUNT-3):
                window = row_array[c:c+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        ## Score Vertical
        for c in range(COLUMN_COUNT):
            col_array = [int(i) for i in list(board[:,c])]
            for r in range(ROW_COUNT-3):
                window = col_array[r:r+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        '''
        Score positive sloped diagonal
        Here, it evaluates diagonal lines sloping upwards.
        It moves through the board, creating diagonal windows and scoring them.
        '''
        ## [TODO-1]
        for x in range(ROW_COUNT - 3):
          for y in range(COLUMN_COUNT - 3):
            window = [board[x + i][y + i] for i in range(WINDOW_LENGTH)]
            score += self.evaluate_window(window, piece)

        '''
        This part evaluates diagonals sloping downwards,
        again using evaluate_window to score these sections.
        '''
        ## [TODO-2]
        for h in range(ROW_COUNT - 3):
          for k in range(COLUMN_COUNT - 3):
            window = [board[h + 3 - i][k + i] for i in range(WINDOW_LENGTH)]
            score += self.evaluate_window(window, piece)

        # return scor

        return score

    # check the gaming is over: win/lose/tie
    def is_terminal_node(self, board):
        return self.winning_move(board, PLAYER_PIECE) or self.winning_move(board, AI_PIECE) or len(self.get_valid_locations(board)) == 0

    # find the valid position for current board
    def get_valid_locations(self, board):
        valid_locations = []
        for col in range(COLUMN_COUNT):
            if self.is_valid_location(board, col):
                valid_locations.append(col)
        return valid_locations

    # find the best move of AI/Player according to the current board
    def pick_best_move(self, board, piece):

        valid_locations = self.get_valid_locations(board)
        best_score = -10000
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = self.get_next_open_row(board, col)
            temp_board = board.copy()
            self.drop_piece(temp_board, row, col, piece)
            score = self.score_position(temp_board, piece)
            if score > best_score:
                best_score = score
                best_col = col

        return best_col


In [6]:
# Create the connect-four class and intialize the board
ai = ConnectFour()
board = ai.create_board()
ai.print_board(board)

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0


In [7]:
# Random drop the piece for AI and Player

ai.drop_piece(board, 5, 3, AI_PIECE)
ai.drop_piece(board, 5, 2, AI_PIECE)

ai.drop_piece(board, 4, 3, PLAYER_PIECE)
ai.drop_piece(board, 4, 2, PLAYER_PIECE)
ai.print_board(board)


0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	1.0	1.0	0.0	0.0	0.0
0.0	0.0	2.0	2.0	0.0	0.0	0.0


In [8]:
# get the valid column for current board
valid_locations = ai.get_valid_locations(board)
print(valid_locations)

game_over = ai.is_terminal_node(board)
print(game_over)
# ai.pick_best_move(board, piece=AI_PIECE)
pos_score = ai.score_position(board, piece=1)
print(pos_score)

[0, 1, 2, 3, 4, 5, 6]
False
9


### **Task1: Implement minimax**

>  *In this task, you will implement **only** `minimax` functions. Please ensure that you have addressed all of the [TODO] items  before submission.*



**Task 1 [TODO] list :** \

☑️ 　　**[TODO 1]**: Terminal Condition Check. \
☑️ 　　**[TODO 2]**: Keeps track of the highest score found and gives column of the optimal move in Maximizing Player (AI)'s view.\
☑️ 　　**[TODO 3]**: Keeps track of the lowest score found and gives column of the optimal move in Minimizing Player (Human)'s view.





You can refer to the following paramters for clarification:

1.   `board`: The current state of the game board.

2.   `depth`: The current depth of recursion.

5.  `maximizingPlayer`: A boolean indicating if the current player is the maximizer (AI) or the minimizer (human player).

`Return Value`:

The function returns a tuple containing the column number where the piece should be dropped and the score associated with that move.



In [15]:
class ConnectFour_Minimax(ConnectFour):

    def __init__(self):
        super().__init__()

    def minimax(self, board, depth, maximizingPlayer):
        valid_locations = self.get_valid_locations(board)
        is_terminal = self.is_terminal_node(board)
        if depth == 0 or is_terminal:

            ### [TODO-1]
            '''
            If the depth is 0 or the node is terminal (indicating a win, lose, or tie state),
            the function returns the heuristic value of the node.
            Wins and losses have large positive or negative values, respectively.
            '''
            if is_terminal:
                if self.winning_move(board, AI_PIECE):
                    return (None, math.inf)
                elif self.winning_move(board, PLAYER_PIECE):
                    return (None, -math.inf)
                else:
                    return (None, 0)
            else:
                return (None, self.score_position(board, AI_PIECE))


        if maximizingPlayer:

            ### [TODO-2]
            '''
            When the function is evaluating the AI's move, it needs to maximize the score.
            It iterates through all valid locations, simulates dropping an AI piece there,
            and calls minimax recursively with reduced depth.
            The function keeps track of the highest score found.

            Hint:
            1.  you can refer to the function of 'pick_best_move'
            '''
            best_score = -math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                temp_board = board.copy()
                self.drop_piece(temp_board, row, col, AI_PIECE)
                _, score = self.minimax(temp_board, depth - 1, False)
                if score > best_score:
                    best_score = score
                    best_col = col
            return best_col, best_score


        else: # Minimizing player

            ### [TODO-3]

            '''
            For the human player, this part aims to minimize the score.
            It similarly iterates through valid moves, simulates them, and calls minimax recursively,
            this time trying to find and keep the minimum score.

            Hint:
            1.  you can refer to the function of 'pick_best_move'
            '''
            best_score = math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                temp_board = board.copy()
                self.drop_piece(temp_board, row, col, PLAYER_PIECE)
                _, score = self.minimax(temp_board, depth - 1, True)
                if score < best_score:
                    best_score = score
                    best_col = col
            return best_col, best_score






#### Test cases-1: Play Game


Check yourself by playing with the maximizer! Given your choice, are the maximizer's choices expected?

You can set `maximizingPlayer`,  `initial and set the board` to your own values when you play.

\* Note that if you want to check whether your code is right or not, please don't modify the setting or set them back to their original values after playing.\*

In [12]:
game = ConnectFour_Minimax()

board = game.create_board()

## Set the current board to test whether AI/Human decision is correct

game.drop_piece(board, 5, 3, AI_PIECE)
game.drop_piece(board, 5, 2, AI_PIECE)

game.drop_piece(board, 4, 3, PLAYER_PIECE)
game.drop_piece(board, 4, 2, PLAYER_PIECE)
game.print_board(board)
import time
start = time.time()
col, minimax_score = game.minimax(board, 5, False)
end = time.time()
print(f'Running time is: {end-start}s')

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	1.0	1.0	0.0	0.0	0.0
0.0	0.0	2.0	2.0	0.0	0.0	0.0
Running time is: 4.372679948806763s


#### Test cases-2: Get the optimal move


Check yourself by playing with the maximizer!

Get the optimal move and drop it to the current board.

You can set `maximizingPlayer`,  `initial and set the board` to your own values when you play.

Note that if you want to check whether your code is right or not, please don't modify the basic setting or set them back to their original values after playing.

In [13]:

if game.is_valid_location(board, col):
    row = game.get_next_open_row(board, col)
    print(f'The optimal move is {(col, row)}')
print(game.is_valid_location(board, col))
game.drop_piece(board, row, col, PLAYER_PIECE)
game.print_board(board)

The optimal move is (1, 5)
True
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	1.0	1.0	0.0	0.0	0.0
0.0	1.0	2.0	2.0	0.0	0.0	0.0


### **Task2: Implement minimax alpha-beta pruning**

>  *In this task, you will implement **only** `minimax_pruning` function. Please ensure that you have addressed all of the [TODO] items  before submission.*



**Task 1 [TODO] list :**

☑️ 　　**[TODO 1]**: Terminal Condition Check. \
☑️ 　　**[TODO 2]**: Keeps track of the highest score found and gives column of the optimal move in Maximizing Player (AI)'s view using `alpha-beta pruning`.\
☑️ 　　**[TODO 3]**: Keeps track of the lowest score found and gives column of the optimal move in Minimizing Player (Human)'s view using `alpha-beta pruning`.






You can refer to the following paramters for clarification:

1.   `board`: The current state of the game board.

2.   `depth`: The current depth of recursion.

3.   `alpha`: The best already explored option along the path to the root for the maximizer.

4.   `beta`: The best already explored option along the path to the root for the minimizer.

5.  `maximizingPlayer`: A boolean indicating if the current player is the maximizer (AI) or the minimizer (human player).

`Return Value`:

The function returns a tuple containing the column number where the piece should be dropped and the score associated with that move.


In [17]:
class ConnectFour_Minimax_Pruning(ConnectFour):

    def __init__(self):
        super().__init__()

    def minimax_pruning(self, board, depth, alpha, beta, maximizingPlayer):
        valid_locations = self.get_valid_locations(board)
        is_terminal = self.is_terminal_node(board)
        if depth == 0 or is_terminal:

            ### [TODO-1]
            '''
            If the depth is 0 or the node is terminal (indicating a win, lose, or tie state),
            the function returns the heuristic value of the node.
            Wins and losses have large positive or negative values, respectively.
            '''
            if is_terminal:
                if self.winning_move(board, AI_PIECE):
                    return (None, math.inf)
                elif self.winning_move(board, PLAYER_PIECE):
                    return (None, -math.inf)
                else:
                    return (None, 0)
            else:
                return (None, self.score_position(board, AI_PIECE))


        if maximizingPlayer:

            ### [TODO-2]
            '''
            When the function is evaluating the AI's move, it needs to maximize the score.
            It iterates through all valid locations, simulates dropping an AI piece there,
            and calls minimax recursively with reduced depth.
            The function keeps track of the highest score found and uses alpha-beta pruning to
            skip evaluating branches that won't be chosen.

            Hint:
            1.  you can refer to the function of 'pick_best_move'
            '''
            best_score = -math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                temp_board = board.copy()
                self.drop_piece(temp_board, row, col, AI_PIECE)
                _, score = self.minimax_pruning(temp_board, depth - 1, alpha, beta, False)
                if score > best_score:
                    best_score = score
                    best_col = col
                alpha = max(alpha, score)
                if alpha >= beta:
                    break  # Beta cutoff
            return best_col, best_score


        else: # Minimizing player

            ### [TODO-3]

            '''
            For the human player, this part aims to minimize the score.
            It similarly iterates through valid moves, simulates them, and calls minimax recursively,
            this time trying to find and keep the minimum score and apply alpha-beta pruning.

            Hint:
            1.  you can refer to the function of 'pick_best_move'
            '''
            best_score = math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                temp_board = board.copy()
                self.drop_piece(temp_board, row, col, PLAYER_PIECE)
                _, score = self.minimax_pruning(temp_board, depth - 1, alpha, beta, True)
                if score < best_score:
                    best_score = score
                    best_col = col
                beta = min(beta, score)
                if beta <= alpha:
                    break
            return best_col, best_score





#### **Test 3: Running Time**
For the same test board, we can check the running time difference between original minimax and minimax_alpha_beta_pruning.

In [22]:
game = ConnectFour_Minimax_Pruning()

board = game.create_board()

## Set the current board to test whether AI/Human decision is correct

game.drop_piece(board, 5, 3, AI_PIECE)
game.drop_piece(board, 5, 2, AI_PIECE)

game.drop_piece(board, 4, 3, PLAYER_PIECE)
game.drop_piece(board, 4, 2, PLAYER_PIECE)
game.print_board(board)
import time
start = time.time()
col, minimax_score = game.minimax_pruning(board, 5,-math.inf, math.inf, False)
end = time.time()
print(f'Afer using the alpha-beta pruning, running time is: {end-start}s')

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	1.0	1.0	0.0	0.0	0.0
0.0	0.0	2.0	2.0	0.0	0.0	0.0
Afer using the alpha-beta pruning, running time is: 0.45800280570983887s
